In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [138]:
def create_timeseries(df, period=1):
    for i in range(period):
        df["c_" + str(i+1)] = df.casual.shift(i+1)
        df["c_" + str(i+1)][:i] = 0

    for i in range(period):
        df["r_" + str(i+1)] = df.registered.shift(i+1)
        df["r_" + str(i+1)][:i] = 0
        
    for i in range(period):        
        df["t_" + str(i+1)] = df["c_" + str(i+1)] + df["r_" + str(i+1)]
        df["c_" + str(i+1)][:i] = 0
        
    df.dropna(inplace=True)
    return df 

def add_avg_prior_demand(df, period=2):
    df["avg_casual"] = df.casual.rolling(period).mean()
    df["avg_registered"] = df.registered.rolling(period).mean()

    df["avg_casual"][:period] = df["avg_casual"][period] 
    df["avg_registered"][:period] = df["avg_registered"][period]
    return df 

def add_avg_prev_day_demand(df):
    for i in range(len(df)):
        if df.hr == 0:
            
            yr = df.yr[i]
            mnth = df.mnth[i]
            day = df.day[i]
            
            if i == 0:
                
            else:
            
    

df = pd.read_csv("hour.csv")
df["day"] = df.dteday.str[-2:].astype("int")

df = create_timeseries(df)

# df_train = df[df.day <= 19]
# df_test = df[df.day > 19]

df_train = df[df.yr == 0]
df_test = df[df.yr == 1]

display(df_test.head())

df_train.to_csv("F:/Data/AzureML/bike_rental_yr0_train.csv", index=False)
df_test.to_csv("F:/Data/AzureML/bike_rental_yr1_test.csv", index=False)

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,...,atemp,hum,windspeed,casual,registered,cnt,day,casual_1,registered_1,cnt_1
8645,8646,2012-01-01,1,1,1,0,0,0,0,1,...,0.3788,0.66,0.0000,5,43,48,1,4.0,27.0,31.0
8646,8647,2012-01-01,1,1,1,1,0,0,0,1,...,0.3485,0.66,0.1343,15,78,93,1,5.0,43.0,48.0
8647,8648,2012-01-01,1,1,1,2,0,0,0,1,...,0.3485,0.76,0.0000,16,59,75,1,15.0,78.0,93.0
8648,8649,2012-01-01,1,1,1,3,0,0,0,1,...,0.3333,0.81,0.0000,11,41,52,1,16.0,59.0,75.0
8649,8650,2012-01-01,1,1,1,4,0,0,0,1,...,0.3030,0.81,0.0896,0,8,8,1,11.0,41.0,52.0


### Load Data

In [140]:
# df_train = pd.read_csv("F:/Data/AzureML/bike_rental_train.csv")
# df_test = pd.read_csv("F:/Data/AzureML/bike_rental_test.csv")

df_train = pd.read_csv("F:/Data/AzureML/bike_rental_yr0_train.csv")
df_test = pd.read_csv("F:/Data/AzureML/bike_rental_yr1_test.csv")

print(df_train.shape)
print(df_test.shape)

display(df_train.head())
print(df_train.columns)

(8644, 21)
(8734, 21)


,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,...,atemp,hum,windspeed,casual,registered,cnt,day,casual_1,registered_1,cnt_1
0,2,2011-01-01,1,0,1,1,0,6,0,1,...,0.2727,0.80,0.0000,8,32,40,1,3.0,13.0,16.0
1,3,2011-01-01,1,0,1,2,0,6,0,1,...,0.2727,0.80,0.0000,5,27,32,1,8.0,32.0,40.0
2,4,2011-01-01,1,0,1,3,0,6,0,1,...,0.2879,0.75,0.0000,3,10,13,1,5.0,27.0,32.0
3,5,2011-01-01,1,0,1,4,0,6,0,1,...,0.2879,0.75,0.0000,0,1,1,1,3.0,10.0,13.0
4,6,2011-01-01,1,0,1,5,0,6,0,2,...,0.2576,0.75,0.0896,0,1,1,1,0.0,1.0,1.0


Index(['instant', 'dteday', 'season', 'yr', 'mnth', 'hr', 'holiday', 'weekday',
       'workingday', 'weathersit', 'temp', 'atemp', 'hum', 'windspeed',
       'casual', 'registered', 'cnt', 'day', 'casual_1', 'registered_1',
       'cnt_1'],
      dtype='object')


In [141]:
from sklearn.preprocessing import MinMaxScaler

def one_hot_encode_df(df, cols):
    for c in cols:
        df = pd.concat([df, pd.get_dummies(df[c], prefix=c)], axis=1)

    return df.drop(cols,axis=1)


x_cols = ['season', 'yr', 'mnth', 'hr', 'holiday', 'weekday',
       'workingday', 'weathersit', 'temp', 'atemp', 'hum', 'windspeed',
       'casual_1', 'registered_1', 'cnt_1']

x_train = df_train[x_cols]
y_train  = df_train[['cnt']]
y_train_casual = df_train[['casual']]
y_train_reg = df_train[['registered']]

x_test = df_test[x_cols]
y_test = df_test[['cnt']]
y_test_casual = df_test[['casual']]
y_test_reg = df_test[['registered']]

x_train = one_hot_encode_df(x_train,['season', 'mnth', 'hr', 'holiday', 'weekday', 'workingday','weathersit'])
x_test = one_hot_encode_df(x_test,['season', 'mnth', 'hr', 'holiday', 'weekday', 'workingday','weathersit'])

display(x_train.head())
print(x_train.columns)


,yr,temp,atemp,hum,windspeed,casual_1,registered_1,cnt_1,season_1,season_2,...,weekday_3,weekday_4,weekday_5,weekday_6,workingday_0,workingday_1,weathersit_1,weathersit_2,weathersit_3,weathersit_4
0,0,0.22,0.2727,0.80,0.0000,3.0,13.0,16.0,1,0,...,0,0,0,1,1,0,1,0,0,0
1,0,0.22,0.2727,0.80,0.0000,8.0,32.0,40.0,1,0,...,0,0,0,1,1,0,1,0,0,0
2,0,0.24,0.2879,0.75,0.0000,5.0,27.0,32.0,1,0,...,0,0,0,1,1,0,1,0,0,0
3,0,0.24,0.2879,0.75,0.0000,3.0,10.0,13.0,1,0,...,0,0,0,1,1,0,1,0,0,0
4,0,0.24,0.2576,0.75,0.0896,0.0,1.0,1.0,1,0,...,0,0,0,1,1,0,0,1,0,0


Index(['yr', 'temp', 'atemp', 'hum', 'windspeed', 'casual_1', 'registered_1',
       'cnt_1', 'season_1', 'season_2', 'season_3', 'season_4', 'mnth_1',
       'mnth_2', 'mnth_3', 'mnth_4', 'mnth_5', 'mnth_6', 'mnth_7', 'mnth_8',
       'mnth_9', 'mnth_10', 'mnth_11', 'mnth_12', 'hr_0', 'hr_1', 'hr_2',
       'hr_3', 'hr_4', 'hr_5', 'hr_6', 'hr_7', 'hr_8', 'hr_9', 'hr_10',
       'hr_11', 'hr_12', 'hr_13', 'hr_14', 'hr_15', 'hr_16', 'hr_17', 'hr_18',
       'hr_19', 'hr_20', 'hr_21', 'hr_22', 'hr_23', 'holiday_0', 'holiday_1',
       'weekday_0', 'weekday_1', 'weekday_2', 'weekday_3', 'weekday_4',
       'weekday_5', 'weekday_6', 'workingday_0', 'workingday_1',
       'weathersit_1', 'weathersit_2', 'weathersit_3', 'weathersit_4'],
      dtype='object')


In [142]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_log_error

model_casual = RandomForestRegressor(n_estimators = 10)
model_reg = RandomForestRegressor(n_estimators = 10)

model_casual.fit(x_train,y_train_casual)
model_reg.fit(x_train,y_train_reg)



f:\anaconda_data\trading\lib\site-packages\ipykernel_launcher.py:7: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  import sys
f:\anaconda_data\trading\lib\site-packages\ipykernel_launcher.py:8: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

### Inference by Using Previous Hour Actual Value

In [143]:
y_pred_casual = model_casual.predict(x_test)
y_pred_reg = model_reg.predict(x_test)
y_pred = np.round(y_pred_casual + y_pred_reg).squeeze()

rmsle = np.sqrt(mean_squared_log_error(y_test, y_pred))
print(rmsle)

df_out = pd.DataFrame(data={"y_test":y_test.values.squeeze(), "y_pred":y_pred})
display(df_out[:20])

0.35614546954197845


,y_test,y_pred
0,48,27.0
1,93,34.0
2,75,76.0
3,52,57.0
4,8,23.0
5,5,5.0
6,2,6.0
7,7,6.0
8,14,15.0
9,40,32.0


### Inference by Using Previous Day Predicted Value

In [127]:
y_pred = list()

for i in range(len(y_test)):
    y_pred_casual = model_casual.predict(x_test.iloc[i,:].values.reshape(1,-1))
    y_pred_reg = model_reg.predict(x_test.iloc[i,:].values.reshape(1,-1))
    y_pred_cnt = y_pred_casual + y_pred_reg
    y_pred.append(y_pred_cnt)
    
    if i < len(y_test) - 1:
        x_test.iloc[i+1,5] = y_pred_casual
        x_test.iloc[i+1,6] = y_pred_reg
        x_test.iloc[i+1,7] = y_pred_cnt
    

y_pred = np.round(y_pred).squeeze()
rmsle = np.sqrt(mean_squared_log_error(y_test, y_pred))
print(rmsle)

df_out = pd.DataFrame(data={"y_test":y_test.values.squeeze(), "y_pred":y_pred})
display(df_out[:20])

0.6057068829605783


,y_test,y_pred
0,13,3.0
1,5,4.0
2,2,3.0
3,1,2.0
4,1,2.0
5,6,5.0
6,35,33.0
7,101,97.0
8,249,219.0
9,143,130.0
